## End-to-end machine learning application
## Data engineering

This project aims to integrate different aspects of a machine learning system, thus developing an end-to-end ML project. The final product is an app (hypothetically called *AppSafe*) composed of a model that calculates the risk of a mobile app being a malware and an API that could integrate with an app store and with the user by sending him/her a warning message when the mobile app that is about to be downloaded is too risky.

The project follows the traditional [CRISP-DM](https://pt.wikipedia.org/wiki/Cross_Industry_Standard_Process_for_Data_Mining) methodology, so these are the main stages that make the core of the project: data engineering, data preparation, data modeling, and deployment.

-----------

This notebook imports all relevant libraries, custom functions and classes, and the data in order to understand it and to make it ready for being pre-processed and finally modeled so a binary classifier can be constructed to predict whether a given mobile app is a malware.

Consequently, this notebook has a section of [data understanding and cleaning](#data_und_clean)<a href='#data_und_clean'></a>, where the data type of all variables is defined together with their domains, as the quantity and samples of unique values are collected. Next, the number of missing values is assessed and features are classified according to their data types and following an empirical classification that reflects how a mobile app works.

An [exploratory data analysis](#eda)<a href='#eda'></a> section extracts insights from data by calculating several distributions regarding input variables and the binary target. The same sort of statistics are applied after new features are created from original data in a [feature engineering](#feat_eng)<a href='#feat_eng'></a> section.

By the end of this notebook, data is ready to be processed in the appropriate way so the designed solution can be developed. As such, data engineering can be seen here as a collection of ETL (extract, transform and load) operations, except for data understanding and EDA tasks, which have an analytical perspective.

**Summary:**
1. [Libraries](#libraries)<a href='#libraries'></a>.
2. [Functions and classes](#functions_classes)<a href='#functions_classes'></a>.
3. [Settings](#settings)<a href='#settings'></a>.
4. [Data imports](#data_imports)<a href='#data_imports'></a>.
  * [Features and labels](#features_labels)<a href='#features_labels'></a>.

5. [Data understanding and cleaning](#data_und_clean)<a href='#data_und_clean'></a>.
  * [Data types](#data_types)<a href='#data_types'></a>.
  * [Unique values](#unique_values)<a href='#unique_values'></a>.
  * [Missings](#missings)<a href='#missings'></a>.
  * [Features](#features)<a href='#features'></a>.
  * [Data cleaning](#data_cleaning)<a href='#data_cleaning'></a>.


6. [Exploratory data analysis](#eda)<a href='#eda'></a>.
  * [Distribution of labels (P(Y))](#dist_y)<a href='#dist_y'></a>.
  * [Distribution of covariates (P(X))](#dist_x)<a href='#dist_x'></a>.
  * [Distribution of covariates conditional on labels (P(X|Y))](#dist_x_y)<a href='#dist_x_y'></a>.
  * [Distribution of labels conditional on covariates (P(Y|X))](#dist_y_x)<a href='#dist_y_x'></a>.
  

7. [Feature engineering](#feat_eng)<a href='#feat_eng'></a>.
  * [Number of related apps](#num_related_apps)<a href='#num_related_apps'></a>.
  * [Number of words in description](#num_words_desc)<a href='#num_words_desc'></a>.
  * [Share of malware related apps](#share_malware_related_apps)<a href='#share_malware_related_apps'></a>.
  * [Natural language processing](#nlp)<a href='#nlp'></a>.
  * [Exporting training and test data](#export_data)<a href='#export_data'></a>.

<a id='libraries'></a>

## Libraries





In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd "/content/gdrive/MyDrive/Studies/end_to_end_ml/notebooks/"

/content/gdrive/MyDrive/Studies/end_to_end_ml/model_dev


In [ ]:
# !pip install -r ../requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
import time

In [ ]:
import sys

sys.path.append(
    os.path.abspath(
        os.path.join(
            os.path.dirname(__doc__), '../src'
        )
    )
)

<a id='functions_classes'></a>

## Functions and classes

In [ ]:
from utils import train_test_split, correct_col_name
from kfolds import Kfolds_fit
from feat_eng import known_related_apps, related_malwares
from data_vis import plot_bar

<a id='settings'></a>

## Settings

In [ ]:
# Declare whether outcomes should be exported:
EXPORT = False

<a id='data_imports'></a>

## Data imports

<a id='features_labels'></a>

### Features and labels

In [ ]:
df_train = pd.read_csv('../data/Android_Permission.csv')

# Columns names:
df_train.columns = [correct_col_name(c) for c in df_train.columns]

print(f'Shape of df: {df_train.shape}.')

# Removing duplicates:
df_train.drop_duplicates(inplace=True)
print(f'Number of instances after removing duplicates: {len(df_train)}.')

# Creating an id variable for each app:
df_train['app_id'] = [i+1 for i in range(len(df_train))]

# Missings in the response variable:
if df_train['class'].isnull().sum() > 0:
  print('There are missings in the response variable!')

# Missings in the primary key:
if df_train['app_id'].isnull().sum() > 0:
  print('There are missings in the primary key!')

# Auxiliary variables:
drop_vars = ['app', 'package', 'class', 'app_id']

df_train.head(3)

Shape of df: (29999, 184).
Number of instances after removing duplicates: 27310.


,app,package,category,description,rating,number_of_ratings,price,related_apps,dangerous_permissions_count,safe_permissions_count,access_drm_content_,access_email_provider_data,access_all_system_downloads,access_download_manager_,advanced_download_manager_functions_,audio_file_access,install_drm_content_,modify_google_service_configuration,modify_google_settings,move_application_resources,read_google_settings,send_download_notifications_,voice_search_shortcuts,access_surfaceflinger,access_checkin_properties,access_the_cache_filesystem,access_to_passwords_for_google_accounts,act_as_an_account_authenticator,bind_to_a_wallpaper,bind_to_an_input_method,change_screen_orientation,coarse,control_location_update_notifications,control_system_backup_and_restore,delete_applications,delete_other_applications_caches,delete_other_applications_data,directly_call_any_phone_numbers,directly_install_applications,disable_or_modify_status_bar,...,your_accounts_act_as_an_account_authenticator,your_accounts_act_as_the_accountmanagerservice,your_accounts_contacts_data_in_google_accounts,your_accounts_discover_known_accounts,your_accounts_manage_the_accounts_list,your_accounts_read_google_service_configuration,your_accounts_use_the_authentication_credentials_of_an_account,your_accounts_view_configured_accounts,your_location_access_extra_location_provider_commands,your_location_coarse,your_location_fine,your_location_mock_location_sources_for_testing,your_messages_read_email_attachments,your_messages_send_gmail,your_messages_edit_sms_or_mms,your_messages_modify_gmail,your_messages_read_gmail,your_messages_read_gmail_attachment_previews,your_messages_read_sms_or_mms,your_messages_read_instant_messages,your_messages_receive_mms,your_messages_receive_sms,your_messages_receive_wap,your_messages_send_sms_received_broadcast,your_messages_send_wap_push_received_broadcast,your_messages_write_instant_messages,your_personal_information_add_or_modify_calendar_events_and_send_email_to_guests,your_personal_information_choose_widgets,your_personal_information_read_browsers_history_and_bookmarks,your_personal_information_read_calendar_events,your_personal_information_read_contact_data,your_personal_information_read_sensitive_log_data,your_personal_information_read_user_defined_dictionary,your_personal_information_retrieve_system_internal_state,your_personal_information_set_alarm_in_alarm_clock,your_personal_information_write_browsers_history_and_bookmarks,your_personal_information_write_contact_data,your_personal_information_write_to_user_defined_dictionary,class,app_id
0,Canada Post Corporation,com.canadapost.android,Business,Canada Post Mobile App gives you access to som...,3.1,77,0.0,"{com.adaffix.pub.ca.android, com.kevinquan.gas...",7.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
1,Word Farm,com.realcasualgames.words,Brain & Puzzle,Speed and strategy combine in this exciting wo...,4.3,199,0.0,"{air.com.zubawing.FastWordLite, com.joybits.do...",3.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,Fortunes of War FREE,fortunesofwar.free,Cards & Casino,"Fortunes of War is a fast-paced, easy to learn...",4.1,243,0.0,"{com.kevinquan.condado, hu.monsta.pazaak, net....",1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


<a id='data_und_clean'>

## Data understanding and cleaning

<a id='data_types'></a>

### Data types

In [ ]:
# Dataframe with data types:
data_types = pd.DataFrame(df_train.dtypes, columns=['type']).reset_index(drop=False)
data_types.columns = ['feature', 'type']

# Dictionary with data types:
data_types_dict = dict(zip(data_types['feature'], data_types['type']))

print('\033[1mDistribution of data types:\033[0m')
print(data_types.type.value_counts())

Distribution of data types:
int64      177
object       5
float64      3
Name: type, dtype: int64


<a id='unique_values'></a>

### Unique values

In [ ]:
n_unique_df = pd.DataFrame(data={
    'feature': [c for c in df_train.columns],
    'n_unique': [df_train[c].nunique() for c in df_train.columns]
}).sort_values('n_unique', ascending=False)

n_unique_df['sample_values'] = n_unique_df.feature.apply(lambda x: list(df_train[x].unique()) if len(list(df_train[x].unique())) <= 10 else
                                                         np.random.choice(list(df_train[x].unique()), size=10, replace=False))

n_unique_df.head(10)

,feature,n_unique,sample_values
184,app_id,27310,"[8277, 14107, 3219, 5482, 3626, 2618, 95, 1996..."
7,related_apps,23868,"[{com.swarcon.cumin.full, com.skycomuk.android..."
3,description,23552,[Rocket dialer - the most professional dialer ...
1,package,23485,"[org.hou.qoutes.love, com.untappdllc.app, elet..."
0,app,22823,"[IHideUFind-Colors, Crazy Orchid HD Wallpaper,..."
5,number_of_ratings,5312,"[15835, 25371, 1612, 2962, 1438180, 73905, 192..."
6,price,425,"[3.89, 14.75, 4.69, 5.58, 4.82, 1.02, 2.44, 3...."
4,rating,42,"[4.4, 2.2, 2.6, 2.7, 2.4, 4.6, 1.9, 4.2, 4.9, ..."
2,category,30,"[Lifestyle, Sports Games, Libraries & Demo, Ca..."
8,dangerous_permissions_count,28,"[23.0, 22.0, 12.0, 11.0, 10.0, 15.0, 18.0, 7.0..."


#### Unique values of the primary key

In [ ]:
print(f'Number of rows: {len(df_train)}.')
n_unique_df[n_unique_df.feature=='app_id']

Number of rows: 27310.


,feature,n_unique,sample_values
184,app_id,27310,"[8277, 14107, 3219, 5482, 3626, 2618, 95, 1996..."


#### Unique values of the response variable

In [ ]:
n_unique_df[n_unique_df.feature=='class']

,feature,n_unique,sample_values
183,class,2,"[0, 1]"


<a id='missings'></a>

### Missings

In [ ]:
missings_df = pd.DataFrame(data={
    'feature': df_train.isnull().sum().index,
    'num_missings': df_train.isnull().sum().values,
    'share_missings': [v/len(df_train) for v in df_train.isnull().sum().values]
}).sort_values('num_missings', ascending=False)
missings_df.head(10)

,feature,num_missings,share_missings
7,related_apps,720,0.026364
8,dangerous_permissions_count,201,0.007360
3,description,3,0.000110
0,app,1,0.000037
128,system_tools_set_wallpaper_size_hints,0,0.000000
119,system_tools_read_sync_statistics,0,0.000000
120,system_tools_read_write_to_resources_owned_by_...,0,0.000000
121,system_tools_reorder_running_applications,0,0.000000
122,system_tools_retrieve_running_applications,0,0.000000
123,system_tools_send_package_removed_broadcast,0,0.000000


#### Missings by label

In [ ]:
# Observations with y = 0:
missings_y0_df = pd.DataFrame(data={
    'feature': df_train[df_train['class']==0].isnull().sum().index,
    'num_missings_y0': df_train[df_train['class']==0].isnull().sum().values,
    'share_missings_y0': [v/len(df_train[df_train['class']==0]) for v in df_train[df_train['class']==0].isnull().sum().values]
}).sort_values('num_missings_y0', ascending=False)

# Observations with y = 1:
missings_y1_df = pd.DataFrame(data={
    'feature': df_train[df_train['class']==1].isnull().sum().index,
    'num_missings_y1': df_train[df_train['class']==1].isnull().sum().values,
    'share_missings': [v/len(df_train[df_train['class']==1]) for v in df_train[df_train['class']==1].isnull().sum().values]
}).sort_values('num_missings_y1', ascending=False)

missings_by_label_df = missings_y0_df.merge(missings_y1_df, on='feature', how='left').sort_values('num_missings_y1', ascending=False)
missings_by_label_df.head(10)

,feature,num_missings_y0,share_missings_y0,num_missings_y1,share_missings
0,related_apps,81,0.008923,639,0.035048
2,dangerous_permissions_count,3,0.000330,198,0.010860
3,app,0,0.000000,1,0.000055
128,access_download_manager_,0,0.000000,0,0.000000
119,access_checkin_properties,0,0.000000,0,0.000000
120,category,0,0.000000,0,0.000000
121,rating,0,0.000000,0,0.000000
122,number_of_ratings,0,0.000000,0,0.000000
123,price,0,0.000000,0,0.000000
124,safe_permissions_count,0,0.000000,0,0.000000


#### Missings by observation

In [ ]:
missings_rows_df = pd.DataFrame(data={
    'idx_obs': df_train.T.isnull().sum().index,
    'num_missings': df_train.T.isnull().sum().values,
    'share_missings': [v/len(df_train) for v in df_train.T.isnull().sum().values]
}).sort_values('num_missings', ascending=False)
missings_rows_df.head(10)

,idx_obs,num_missings,share_missings
26092,28548,2,0.000073
19011,20322,2,0.000073
11180,11639,2,0.000073
14087,14813,2,0.000073
27059,29694,2,0.000073
27062,29699,2,0.000073
9515,9832,2,0.000073
8751,9017,2,0.000073
7670,7869,2,0.000073
2413,2434,2,0.000073


In [ ]:
display(missings_rows_df.num_missings.describe())
print('\n')
display(missings_rows_df.num_missings.value_counts())

count    27310.000000
mean         0.033870
std          0.217818
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          2.000000
Name: num_missings, dtype: float64

0    26586
1      523
2      201
Name: num_missings, dtype: int64

<a id='features'></a>

### Features

In [ ]:
data_und = data_types.merge(n_unique_df, on='feature', how='left')
data_und = data_und.merge(missings_df, on='feature', how='left')

data_und.sample(10)

,feature,type,n_unique,sample_values,num_missings,share_missings
85,network_communication_download_files_without_n...,int64,1,[0],0,0.0
107,system_tools_expand_collapse_status_bar,int64,2,"[0, 1]",0,0.0
92,phone_calls_modify_phone_state,int64,2,"[0, 1]",0,0.0
127,system_tools_set_wallpaper,int64,2,"[0, 1]",0,0.0
172,your_personal_information_choose_widgets,int64,2,"[0, 1]",0,0.0
35,delete_other_applications_caches,int64,2,"[0, 1]",0,0.0
101,system_tools_change_background_data_usage_setting,int64,1,[0],0,0.0
117,system_tools_read_subscribed_feeds,int64,2,"[0, 1]",0,0.0
169,your_messages_send_wap_push_received_broadcast,int64,2,"[0, 1]",0,0.0
10,access_drm_content_,int64,2,"[0, 1]",0,0.0


#### Features by type

In [ ]:
data_und['var_class'] = ''

Text variables

In [ ]:
data_und[data_und['type']==object]

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class
0,app,object,22823,"[Alabama Crimson Tide News, Blood Demon Movie,...",1,0.000037,
1,package,object,23485,"[com.estrongs.android.pop.app.shortcut, com.tw...",0,0.000000,
2,category,object,30,"[Shopping, Racing, Productivity, Sports Games,...",0,0.000000,
3,description,object,23552,"[Enjoy Navionics??? Anytime, Anywhere.<p>The W...",3,0.000110,
7,related_apps,object,23868,[{com.warting.blogg.wis_trevortransdgtl_feed_n...,720,0.026364,


In [ ]:
cat_vars = list(data_und[data_und['type']==object]['feature'])
data_und.loc[data_und.feature.isin(cat_vars), 'var_class'] = 'categorical'

Binary variables

In [ ]:
data_und[data_und['feature'].isin([c for c in data_und['feature'] if (data_types_dict[c]!=object) & (df_train[c].nunique()==2)])]

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class
10,access_drm_content_,int64,2,"[0, 1]",0,0.0,
11,access_email_provider_data,int64,2,"[0, 1]",0,0.0,
13,access_download_manager_,int64,2,"[0, 1]",0,0.0,
14,advanced_download_manager_functions_,int64,2,"[0, 1]",0,0.0,
15,audio_file_access,int64,2,"[0, 1]",0,0.0,
...,...,...,...,...,...,...,...
179,your_personal_information_set_alarm_in_alarm_c...,int64,2,"[0, 1]",0,0.0,
180,your_personal_information_write_browsers_histo...,int64,2,"[0, 1]",0,0.0,
181,your_personal_information_write_contact_data,int64,2,"[1, 0]",0,0.0,
182,your_personal_information_write_to_user_define...,int64,2,"[0, 1]",0,0.0,


In [ ]:
binary_vars = list(data_und[data_und['feature'].isin([c for c in data_und['feature'] if (data_types_dict[c]!=object) &
                                                      (df_train[c].nunique()==2)])]['feature'])
data_und.loc[data_und.feature.isin(binary_vars), 'var_class'] = 'binary'

Continuous variables

In [ ]:
data_und[data_und['feature'].isin([c for c in data_und['feature'] if (data_types_dict[c]!=object) & (df_train[c].nunique()>2)])]

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class
4,rating,float64,42,"[0.0, 1.9, 1.1, 4.6, 2.5, 3.6, 1.0, 2.9, 4.5, ...",0,0.00000,
5,number_of_ratings,int64,5312,"[6366, 9556, 1937, 2115, 13590, 625, 3685, 377...",0,0.00000,
6,price,float64,425,"[6.78, 1.07, 19.95, 2.41, 1.29, 4.55, 7.95, 4....",0,0.00000,
8,dangerous_permissions_count,float64,28,"[4.0, 13.0, 12.0, 0.0, 22.0, 15.0, 11.0, 10.0,...",201,0.00736,
9,safe_permissions_count,int64,16,"[11, 16, 2, 10, 1, 4, 13, 14, 7, 9]",0,0.00000,
184,app_id,int64,27310,"[2602, 22638, 16825, 23771, 16064, 17568, 2043...",0,0.00000,


In [ ]:
cont_vars = list(data_und[data_und['feature'].isin([c for c in data_und['feature'] if (data_types_dict[c]!=object) &
                                                    (df_train[c].nunique()>2)])]['feature'])
data_und.loc[data_und.feature.isin(cont_vars), 'var_class'] = 'numerical'

In [ ]:
data_und[data_und.var_class=='']

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class
12,access_all_system_downloads,int64,1,[0],0,0.0,
17,modify_google_service_configuration,int64,1,[0],0,0.0,
26,access_to_passwords_for_google_accounts,int64,1,[0],0,0.0,
27,act_as_an_account_authenticator,int64,1,[0],0,0.0,
31,coarse,int64,1,[0],0,0.0,
40,discover_known_accounts,int64,1,[0],0,0.0,
45,full_internet_access,int64,1,[0],0,0.0,
48,mock_location_sources_for_testing,int64,1,[0],0,0.0,
52,modify_delete_usb_storage_contents_modify_dele...,int64,1,[0],0,0.0,
55,permanently_disable_device,int64,1,[0],0,0.0,


#### Features by category

In [ ]:
# Features related with attributes of the app:
app_attributes = [
	"related_apps",
	"price",
	"number_of_ratings",
	"rating",
	"description",
	"category",
	"package",
	"app",
	"app_id"
]

# Features related with actions performed by the app:
actions_calls = list(data_und[(data_und.feature.str.startswith('phone_call')) | (data_und.feature=='directly_call_any_phone_numbers')]['feature'])
actions_write = list(data_und[data_und.feature.str.startswith('write_')]['feature'])
actions_read = list(data_und[data_und.feature.str.startswith('read_')]['feature'])
actions_modify = list(data_und[data_und.feature.str.startswith('modify_')]['feature'])
actions_force = list(data_und[data_und.feature.str.startswith('force_')]['feature'])
actions_control = list(data_und[data_und.feature.str.startswith('control_')]['feature'])
actions_delete = list(data_und[data_und.feature.str.startswith('delete_')]['feature'])
actions_bind = list(data_und[data_und.feature.str.startswith('bind_')]['feature'])
actions_access = list(data_und[data_und.feature.str.startswith('access_')]['feature'])

# Features related with interaction between app and system (device, data, other applications):
interactions_personal_info = list(data_und[data_und.feature.str.startswith('your_personal_info')]['feature'])
interactions_messages = list(data_und[data_und.feature.str.startswith('your_message')]['feature'])
interactions_location = list(data_und[data_und.feature.str.startswith('your_loca')]['feature'])
interactions_accounts = list(data_und[data_und.feature.str.startswith('your_acc')]['feature'])
interactions_sys_tools = list(data_und[data_und.feature.str.startswith('system_tools')]['feature'])
interactions_networks = list(data_und[data_und.feature.str.startswith('network_comm')]['feature'])
interactions_hardware = list(data_und[data_und.feature.str.startswith('hardware_contr')]['feature'])
interactions_dev_tools = list(data_und[data_und.feature.str.startswith('development_tool')]['feature'])

# Other features related with actions performed by the app:
actions_others = [
	"directly_install_applications",
	"dangerous_permissions_count",
	"safe_permissions_count",
	"services_that_cost_you_money_directly_call_phone_numbers",
	"services_that_cost_you_money_send_sms_messages",
	"start_im_service",
	"full_internet_access",
	"enable_or_disable_application_components",
	"manage_application_tokens",
	"prevent_app_switches",
	"update_component_usage_statistics",
	"run_in_factory_test_mode", "coarse", "voice_search_shortcuts",
	"disable_or_modify_status_bar",
	"display_unauthorized_windows",
	"partial_shutdown",
	"power_device_on_or_off",
	"set_time",
	"change_screen_orientation",
	"press_keys_and_control_buttons",
	"send_download_notifications_",
	"permanently_disable_device",
]

# Other features related with interaction between app and system (device, data, other applications):
interactions_others = [
	"storage_modify_delete_usb_storage_contents_modify_delete_sd_card_contents",
	 "set_wallpaper_size_hints",
	"reset_system_to_factory_defaults",
	"record_what_you_type_and_actions_you_take",
	"permission_to_install_a_location_provider",
	"monitor_and_control_all_application_launching",
	"mock_location_sources_for_testing",
	"interact_with_a_device_admin",
	"act_as_an_account_authenticator",
	"move_application_resources",
	"install_drm_content_",
	"audio_file_access",
	"advanced_download_manager_functions_",
	"discover_known_accounts"
]

# Categories of features:
categories_feat = ["app_attributes",
"actions_calls",
"actions_write",
"actions_read",
"actions_modify",
"actions_force",
"actions_control",
"actions_delete",
"actions_bind",
"actions_access",
"interactions_personal_info",
"interactions_messages",
"interactions_location",
"interactions_accounts",
"interactions_sys_tools",
"interactions_networks",
"interactions_hardware",
"interactions_dev_tools",
"actions_others",
"interactions_others"
]

In [ ]:
categories_feat_dict = {}
categories_feat_dict['class'] = 'target'

# Loop over categories:
for c in categories_feat:
  # Loop over features:
  for f in eval(c):
    categories_feat_dict[f] = c

# Category of each variable:
data_und['category'] = data_und.feature.apply(lambda x: categories_feat_dict[x])

In [ ]:
if EXPORT:
    data_und.to_csv('../data/features.csv', index=False)
    
    # Original data:
    input_data = pd.read_csv('../data/Android_Permission.csv')
    input_data.columns = [correct_col_name(c) for c in input_data.columns]

    # Schema of original data:
    schema = dict(
        zip(
            [c for c in input_data.drop(['class'], axis=1).columns],
            ['str' if type(input_data[c].iloc[0])==str else 'numeric' for c in input_data.drop(['class'], axis=1).columns]
        )
    )

<a id='data_cleaning'></a>

### Data cleaning

In [ ]:
# Removing signs from the text of related apps:
df_train['related_apps'] = df_train['related_apps'].apply(lambda x: x if pd.isna(x) else x.replace('{', '').replace('}', ''))

#### Train-test split

In [ ]:
df_train, df_test = train_test_split(df_train, test_ratio=0.33, shuffle=True)

<a id='eda'></a>

## Exploratory data analysis

In [ ]:
print('\033[1mDistribution of features by category:\033[0m')
data_und.category.value_counts()

Distribution of features by category:


interactions_sys_tools        35
actions_others                23
interactions_accounts         21
interactions_messages         14
interactions_others           14
interactions_personal_info    12
interactions_networks          9
app_attributes                 9
actions_access                 8
interactions_hardware          6
actions_modify                 6
interactions_location          4
actions_calls                  4
actions_read                   4
interactions_dev_tools         4
actions_delete                 3
actions_control                2
actions_write                  2
actions_force                  2
actions_bind                   2
target                         1
Name: category, dtype: int64

<a id='dist_y'></a>

### Distribution of labels (P(Y))

In [ ]:
df_train['class'].value_counts()/len(df_train)

1    0.667122
0    0.332878
Name: class, dtype: float64

<a id='dist_x'></a>

### Distribution of covariates (P(X))

#### Category

In [ ]:
df_train['category'].value_counts().head(10)/len(df_train)

Entertainment        0.095530
Travel & Local       0.072303
Books & Reference    0.067931
Arcade & Action      0.060826
Brain & Puzzle       0.059515
Casual               0.054214
Personalization      0.052738
Lifestyle            0.048912
Tools                0.044431
Education            0.043775
Name: category, dtype: float64

#### Rating

In [ ]:
df_train['rating'].describe()

count    18298.000000
mean         3.500628
std          1.451806
min          0.000000
25%          3.300000
50%          4.000000
75%          4.400000
max          5.000000
Name: rating, dtype: float64

In [ ]:
df_train['number_of_ratings'].describe()

count    1.829800e+04
mean     5.602205e+03
std      4.641166e+04
min      0.000000e+00
25%      4.000000e+00
50%      3.800000e+01
75%      4.920000e+02
max      1.908590e+06
Name: number_of_ratings, dtype: float64

#### Price

In [ ]:
df_train['price'].describe()

count    18298.000000
mean         0.664250
std          3.406534
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        149.990000
Name: price, dtype: float64

#### Interaction with system tools

In [ ]:
df_train[list(data_und[data_und.category=='interactions_sys_tools']['feature'])].mean().sort_values(ascending=False)

system_tools_prevent_device_from_sleeping             0.195868
system_tools_automatically_start_at_boot              0.092251
system_tools_retrieve_running_applications            0.048202
system_tools_modify_global_system_settings            0.042136
system_tools_set_wallpaper                            0.037928
system_tools_change_wi_fi_state                       0.028473
system_tools_kill_background_processes                0.022953
system_tools_disable_keylock                          0.021368
system_tools_change_network_connectivity              0.017925
system_tools_bluetooth_administration                 0.013991
system_tools_mount_and_unmount_filesystems            0.011094
system_tools_display_system_level_alerts              0.009291
system_tools_read_sync_settings                       0.008689
system_tools_send_sticky_broadcast                    0.008307
system_tools_write_sync_settings                      0.007979
system_tools_change_your_ui_settings                  0

#### Interaction with accounts

In [ ]:
df_train[list(data_und[data_und.category=='interactions_accounts']['feature'])].mean().sort_values(ascending=False)

your_accounts_discover_known_accounts                             0.057438
your_accounts_use_the_authentication_credentials_of_an_account    0.012898
your_accounts_manage_the_accounts_list                            0.010821
your_accounts_act_as_an_account_authenticator                     0.006613
your_accounts_view_configured_accounts                            0.003990
your_accounts_act_as_the_accountmanagerservice                    0.001749
your_accounts_read_google_service_configuration                   0.001694
your_accounts_access_other_google_services                        0.001585
your_accounts_google_mail                                         0.000601
your_accounts_youtube_usernames                                   0.000383
your_accounts_youtube                                             0.000383
your_accounts_google_spreadsheets                                 0.000328
your_accounts_google_maps                                         0.000328
your_accounts_google_docs

#### Interaction with personal information

In [ ]:
df_train[list(data_und[data_und.category=='interactions_personal_info']['feature'])].mean().sort_values(ascending=False)

your_personal_information_read_contact_data                                         0.084709
your_personal_information_write_contact_data                                        0.037490
your_personal_information_read_sensitive_log_data                                   0.022625
your_personal_information_read_browsers_history_and_bookmarks                       0.017762
your_personal_information_read_calendar_events                                      0.014428
your_personal_information_write_browsers_history_and_bookmarks                      0.014264
your_personal_information_add_or_modify_calendar_events_and_send_email_to_guests    0.013225
your_personal_information_choose_widgets                                            0.001476
your_personal_information_write_to_user_defined_dictionary                          0.001202
your_personal_information_read_user_defined_dictionary                              0.001148
your_personal_information_set_alarm_in_alarm_clock                    

#### Interaction with network resources

In [ ]:
df_train[list(data_und[data_und.category=='interactions_networks']['feature'])].mean().sort_values(ascending=False)

network_communication_full_internet_access                        0.808012
network_communication_view_network_state                          0.554979
network_communication_view_wi_fi_state                            0.133840
network_communication_receive_data_from_internet                  0.039786
network_communication_create_bluetooth_connections                0.022243
network_communication_control_near_field_communication            0.001421
network_communication_make_receive_internet_calls                 0.000273
network_communication_broadcast_data_messages_to_applications_    0.000055
network_communication_download_files_without_notification         0.000000
dtype: float64

#### Interaction with the hardware

In [ ]:
df_train[list(data_und[data_und.category=='interactions_hardware']['feature'])].mean().sort_values(ascending=False)

hardware_controls_control_vibrator              0.221062
hardware_controls_take_pictures_and_videos      0.065417
hardware_controls_record_audio                  0.038419
hardware_controls_change_your_audio_settings    0.027981
hardware_controls_control_flashlight            0.015029
hardware_controls_test_hardware                 0.001366
dtype: float64

#### Interaction with develoment tools

In [ ]:
df_train[list(data_und[data_und.category=='interactions_dev_tools']['feature'])].mean().sort_values(ascending=False)

development_tools_send_linux_signals_to_applications        0.000164
development_tools_enable_application_debugging              0.000109
development_tools_make_all_background_applications_close    0.000055
development_tools_limit_number_of_running_processes         0.000000
dtype: float64

<a id='dist_x_y'></a>

### Distribution of covariates conditional on labels (P(X|Y))

#### Rating

In [ ]:
df_train.groupby('class').describe()[['rating']]

rating                                             
         count      mean       std  min  25%  50%  75%  max
class                                                      
0       6091.0  3.965441  0.696474  0.0  3.7  4.1  4.4  5.0
1      12207.0  3.268698  1.660093  0.0  2.9  3.9  4.4  5.0

In [ ]:
df_train.groupby('class').describe()[['number_of_ratings']]

number_of_ratings                             ...                          
                  count         mean           std  ...    50%     75%        max
class                                               ...                          
0                6091.0  9031.736661  60879.754529  ...  240.0  1825.5  1897622.0
1               12207.0  3890.951503  37025.969904  ...   12.0   155.5  1908590.0

[2 rows x 8 columns]

#### Price

In [ ]:
df_train.groupby('class').describe()[['price']]

price                                                 
         count      mean       std  min  25%  50%   75%     max
class                                                          
0       6091.0  0.000000  0.000000  0.0  0.0  0.0  0.00    0.00
1      12207.0  0.995695  4.131012  0.0  0.0  0.0  0.99  149.99

<a id='dist_y_x'></a>

### Distribution of labels conditional on covariates (P(Y|X))

#### Category

In [ ]:
df_train.groupby('category').mean()[['class']].sort_values('class', ascending=False).head(10)

,class
category,
Transportation,0.985240
Medical,0.983051
Travel & Local,0.981859
Sports,0.956186
News & Magazines,0.928082
Shopping,0.926606
Photography,0.890710
Tools,0.884379
Music & Audio,0.803797


#### Interaction with system tools

In [ ]:
sel_vars = list(data_und[data_und.category=='interactions_sys_tools']['feature'])

# Loop over variables:
for v in sel_vars:
  display(df_train.groupby(v).mean()[['class']].sort_values('class', ascending=False))

,class
system_tools_allow_wi_fi_multicast_reception,
0,0.667233
1,0.625000


,class
system_tools_automatically_start_at_boot,
0,0.670921
1,0.629739


,class
system_tools_bluetooth_administration,
0,0.66772
1,0.62500


,class
system_tools_change_wi_fi_state,
1,0.704415
0,0.666029


,class
system_tools_change_background_data_usage_setting,
0,0.667122


...

,class
system_tools_set_wallpaper,
0,0.673086
1,0.515850


,class
system_tools_set_wallpaper_size_hints,
0,0.667325
1,0.606557


,class
system_tools_write_access_point_name_settings,
1,0.693878
0,0.667050


,class
system_tools_write_subscribed_feeds,
0,0.667414
1,0.352941


,class
system_tools_write_sync_settings,
0,0.668852
1,0.452055


#### Interaction with accounts

In [ ]:
sel_vars = list(data_und[data_und.category=='interactions_accounts']['feature'])

# Loop over variables:
for v in sel_vars:
  display(df_train.groupby(v).mean()[['class']].sort_values('class', ascending=False))

,class
your_accounts_blogger,
0,0.667122


,class
your_accounts_google_app_engine,
0,0.667177
1,0.333333


,class
your_accounts_google_docs,
0,0.667122
1,0.666667


,class
your_accounts_google_finance,
0,0.667195
1,0.000000


,class
your_accounts_google_maps,
0,0.667177
1,0.500000


...

,class
your_accounts_discover_known_accounts,
0,0.674610
1,0.544244


,class
your_accounts_manage_the_accounts_list,
0,0.669337
1,0.464646


,class
your_accounts_read_google_service_configuration,
0,0.667543
1,0.419355


,class
your_accounts_use_the_authentication_credentials_of_an_account,
0,0.668365
1,0.572034


,class
your_accounts_view_configured_accounts,
0,0.667654
1,0.534247


#### Interaction with personal information

In [ ]:
sel_vars = list(data_und[data_und.category=='interactions_personal_info']['feature'])

# Loop over variables:
for v in sel_vars:
  display(df_train.groupby(v).mean()[['class']].sort_values('class', ascending=False))

,class
your_personal_information_add_or_modify_calendar_events_and_send_email_to_guests,
1,0.747934
0,0.666039


,class
your_personal_information_choose_widgets,
1,0.740741
0,0.667013


,class
your_personal_information_read_browsers_history_and_bookmarks,
0,0.671452
1,0.427692


,class
your_personal_information_read_calendar_events,
1,0.753788
0,0.665853


,class
your_personal_information_read_contact_data,
0,0.670886
1,0.626452


...

,class
your_personal_information_retrieve_system_internal_state,
0,0.667268
1,0.400000


,class
your_personal_information_set_alarm_in_alarm_clock,
0,0.667141
1,0.636364


,class
your_personal_information_write_browsers_history_and_bookmarks,
0,0.671453
1,0.367816


,class
your_personal_information_write_contact_data,
0,0.669032
1,0.618076


,class
your_personal_information_write_to_user_defined_dictionary,
0,0.667159
1,0.636364


#### Interaction with network resources

In [ ]:
sel_vars = list(data_und[data_und.category=='interactions_networks']['feature'])

# Loop over variables:
for v in sel_vars:
  display(df_train.groupby(v).mean()[['class']].sort_values('class', ascending=False))

,class
network_communication_broadcast_data_messages_to_applications_,
1,1.000000
0,0.667104


,class
network_communication_control_near_field_communication,
0,0.66736
1,0.50000


,class
network_communication_create_bluetooth_connections,
0,0.667151
1,0.665848


,class
network_communication_download_files_without_notification,
0,0.667122


,class
network_communication_full_internet_access,
0,0.755480
1,0.646128


,class
network_communication_make_receive_internet_calls,
1,0.800000
0,0.667086


,class
network_communication_receive_data_from_internet,
0,0.667786
1,0.651099


,class
network_communication_view_wi_fi_state,
1,0.671703
0,0.666414


,class
network_communication_view_network_state,
0,0.744811
1,0.604825


#### Interaction with the hardware

In [ ]:
sel_vars = list(data_und[data_und.category=='interactions_hardware']['feature'])

# Loop over variables:
for v in sel_vars:
  display(df_train.groupby(v).mean()[['class']].sort_values('class', ascending=False))

,class
hardware_controls_change_your_audio_settings,
0,0.667491
1,0.654297


,class
hardware_controls_control_flashlight,
0,0.667702
1,0.629091


,class
hardware_controls_control_vibrator,
0,0.680138
1,0.621261


,class
hardware_controls_record_audio,
0,0.668315
1,0.637269


,class
hardware_controls_take_pictures_and_videos,
1,0.698413
0,0.664932


,class
hardware_controls_test_hardware,
0,0.667159
1,0.640000


#### Interaction with develoment tools

In [ ]:
sel_vars = list(data_und[data_und.category=='interactions_dev_tools']['feature'])

# Loop over variables:
for v in sel_vars:
  display(df_train.groupby(v).mean()[['class']].sort_values('class', ascending=False))

,class
development_tools_enable_application_debugging,
0,0.66714
1,0.50000


,class
development_tools_limit_number_of_running_processes,
0,0.667122


,class
development_tools_make_all_background_applications_close,
1,1.000000
0,0.667104


,class
development_tools_send_linux_signals_to_applications,
0,0.667177
1,0.333333


#### Bivariate modeling

In [ ]:
features, test_roc_auc, coefs = [], [], []

# Grid of hyper-parameters:
grid_param = {'C': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.25, 0.3, 0.5, 0.75, 1, 3, 10]}
default_param = {'C': 1.0}
fixed_params = {'penalty':'l1', 'solver':'liblinear', 'warm_start':True, 'max_iter':500}

# Loop over variables:
for v in list(data_und[~(data_und.feature.isin(drop_vars)) & (data_und['type']!=object)]['feature']):
  # Creating K-folds CV object:
  kfolds = Kfolds_fit(task='classification', method='logistic_regression', num_folds=5, metric='roc_auc',
                      shuffle=False,
                      random_search=False,
                      grid_param=grid_param, default_param=default_param, fixed_params=fixed_params,
                      pre_selecting=False,
                      parallelize=False)

  # Running K-folds CV:
  kfolds.fit(train_inputs=df_train.dropna()[[v]], train_output=df_train.dropna()['class'],
            test_inputs=df_test.dropna()[[v]], test_output=df_test.dropna()['class'])

  # Collecting outcomes:
  features.append(v)
  test_roc_auc.append(kfolds.performance_metrics['test_roc_auc'])
  coefs.append(kfolds.model.coef_)

---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 5.
   Estimation method: logistic regression.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 0.003}.
   CV performance metric associated with best hyper-parameters: 0.5725.


Performance metrics evaluated at test data:
   test_roc_auc = 0.5689
   test_prec_avg = 0.774
   test_brier = 0.2157
---------------------------------------------------------------------


------------------------------------
Running time: 0.08 minutes.
Start time: 2022-01-29, 17:58:14
End time: 2022-01-29, 17:58:19
------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 5.
   Estimation method: logistic regression.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 0.0001}.
   CV performance metric associated with best hyper-parameters: 0.7342.


Performance metrics evaluated at test data:
   test_roc_auc = 0.7351
   test_prec_avg = 0.8605
   test_brier = 0.25
---------------------------------------------------------------------


------------------------------------
Running time: 0.92 minutes.
Start time: 2022-01-29, 17:58:19
End time: 2022-01-29, 17:59:14
------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 5.
   Estimation method: logistic regression.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 0.0003}.
   CV performance metric associated with best hyper-parameters: 0.6382.


Performance metrics evaluated at test data:
   test_roc_auc = 0.6367
   test_prec_avg = 0.762
   test_brier = 0.2404
---------------------------------------------------------------------


------------------------------------
Running time: 5.44 minutes.
Start time: 2022-01-29, 17:59:15
End time: 2022-01-29, 18:04:41
------------------------------------


...

---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 5.
   Estimation method: logistic regression.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 0.0001}.
   CV performance metric associated with best hyper-parameters: 0.5.


Performance metrics evaluated at test data:
   test_roc_auc = 0.5
   test_prec_avg = 0.6724
   test_brier = 0.25
---------------------------------------------------------------------


------------------------------------
Running time: 0.05 minutes.
Start time: 2022-01-29, 18:13:53
End time: 2022-01-29, 18:13:56
------------------------------------


---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 5.
   Estimation method: logistic regression.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 0.0001}.
   CV performance metric associated with best hyper-parameters: 0.5.


Performance metrics evaluated at test data:
   test_roc_auc = 0.5
   test_prec_avg = 0.6724
   test_brier = 0.25
---------------------------------------------------------------------


------------------------------------
Running time: 0.05 minutes.
Start time: 2022-01-29, 18:13:56
End time: 2022-01-29, 18:13:59
------------------------------------


---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 5.
   Estimation method: logistic regression.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 0.03}.
   CV performance metric associated with best hyper-parameters: 0.5092.


Performance metrics evaluated at test data:
   test_roc_auc = 0.5107
   test_prec_avg = 0.6772
   test_brier = 0.2195
---------------------------------------------------------------------


------------------------------------
Running time: 0.05 minutes.
Start time: 2022-01-29, 18:13:59
End time: 2022-01-29, 18:14:02
------------------------------------


---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 5.
   Estimation method: logistic regression.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 0.1}.
   CV performance metric associated with best hyper-parameters: 0.5036.


Performance metrics evaluated at test data:
   test_roc_auc = 0.5034
   test_prec_avg = 0.6739
   test_brier = 0.2205
---------------------------------------------------------------------


------------------------------------
Running time: 0.05 minutes.
Start time: 2022-01-29, 18:14:02
End time: 2022-01-29, 18:14:05
------------------------------------


---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 5.
   Estimation method: logistic regression.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 0.0001}.
   CV performance metric associated with best hyper-parameters: 0.5.


Performance metrics evaluated at test data:
   test_roc_auc = 0.5
   test_prec_avg = 0.6724
   test_brier = 0.25
---------------------------------------------------------------------


------------------------------------
Running time: 0.05 minutes.
Start time: 2022-01-29, 18:14:06
End time: 2022-01-29, 18:14:08
------------------------------------


In [ ]:
indiv_modeling = pd.DataFrame(data={
    'feature': features, 'test_roc_auc': test_roc_auc, 'coefs': [c[0][0] for c in coefs]
}).sort_values('test_roc_auc', ascending=False)
indiv_modeling.head(10)

,feature,test_roc_auc,coefs
1,number_of_ratings,0.735114,-7.181053e-08
2,price,0.636725,8.390360e-02
85,network_communication_view_network_state,0.574346,-4.501545e-01
0,rating,0.568921,-1.237575e-01
4,safe_permissions_count,0.562917,-3.663021e-02
88,phone_calls_read_phone_state_and_identity,0.538968,-1.249008e-01
81,network_communication_full_internet_access,0.537204,-1.659488e-01
3,dangerous_permissions_count,0.534299,-1.500169e-02
150,your_location_fine,0.531454,2.409798e-01
111,system_tools_prevent_device_from_sleeping,0.518462,-8.572300e-02


In [ ]:
plot_bar(data=indiv_modeling.head(10), x=['feature'], y=['test_roc_auc'], pos=[(1,1)],
             titles=['Test ROC-AUC of individual modeling'], width=900, height=600)

In [ ]:
plot_bar(data=indiv_modeling.head(10), x=['feature'], y=['coefs'], pos=[(1,1)],
         titles=['Coefficient of individual modeling'], width=900, height=600)

In [ ]:
if EXPORT:
  indiv_modeling.to_csv('../experiments/indiv_modeling.csv', index=False)

<a id='feat_eng'></a>

## Feature engineering

<a id='num_related_apps'></a>

### Number of related apps

In [ ]:
# Creating the variable with the number of related apps:
df_train['num_related_apps'] = df_train['related_apps'].apply(lambda x: np.NaN if pd.isna(x) else len(x.split(', ')))
df_test['num_related_apps'] = df_test['related_apps'].apply(lambda x: np.NaN if pd.isna(x) else len(x.split(', ')))

# Updating the list of auxiliary variables:
drop_vars.append('related_apps')

In [ ]:
new_var = 'num_related_apps'

# Describing the new feature:
data_und_update = pd.DataFrame(data={
    'feature': new_var, 'type': df_train.dtypes[new_var], 'n_unique': df_train[new_var].nunique(),
    'sample_values': str(list(df_train[new_var].unique()) if len(list(df_train[new_var].unique())) <= 10 else np.random.choice(list(df_train[new_var].unique()),
                                                                                                                               size=10,
                                                                                                                               replace=False)),
    'num_missings': df_train[new_var].isnull().sum(), 'share_missings': df_train[new_var].isnull().sum()/len(df_train),
    'var_class': 'numerical', 'category': 'app_attributes'
}, index=[0])

# Updating the data understanding dataframe:
data_und = pd.concat([data_und, data_und_update], axis=0, sort=False)
data_und.tail(1)

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class,category
0,num_related_apps,float64,4,"[4.0, 1.0, nan, 3.0, 2.0]",484,0.026451,numerical,app_attributes


#### P(X)

In [ ]:
df_train['num_related_apps'].describe()

count    17814.000000
mean         3.926069
std          0.418756
min          1.000000
25%          4.000000
50%          4.000000
75%          4.000000
max          4.000000
Name: num_related_apps, dtype: float64

In [ ]:
share_miss_new = df_train.num_related_apps.isnull().sum()/len(df_train)
print(f'Share of missings: {share_miss_new:.4f}.')

Share of missings: 0.0265.


#### P(X|Y)

In [ ]:
df_train.groupby('class').describe()[['num_related_apps']]

num_related_apps                                             
                 count      mean       std  min  25%  50%  75%  max
class                                                              
0               6040.0  3.930464  0.409523  1.0  4.0  4.0  4.0  4.0
1              11774.0  3.923815  0.423415  1.0  4.0  4.0  4.0  4.0

<a id='num_words_desc'></a>

### Number of words in description

In [ ]:
# Creating the variable that indicates the number of words in a description:
df_train['num_words_desc'] = df_train.description.apply(lambda x: x if pd.isna(x) else len(x.split(' ')))
df_test['num_words_desc'] = df_test.description.apply(lambda x: x if pd.isna(x) else len(x.split(' ')))

# Updating the list of auxiliary variables:
drop_vars.append('description')

In [ ]:
new_var = 'num_words_desc'

# Describing the new feature:
data_und_update = pd.DataFrame(data={
    'feature': new_var, 'type': df_train.dtypes[new_var], 'n_unique': df_train[new_var].nunique(),
    'sample_values': str(list(df_train[new_var].unique()) if len(list(df_train[new_var].unique())) <= 10 else np.random.choice(list(df_train[new_var].unique()),
                                                                                                                               size=10,
                                                                                                                               replace=False)),
    'num_missings': df_train[new_var].isnull().sum(), 'share_missings': df_train[new_var].isnull().sum()/len(df_train),
    'var_class': 'numerical', 'category': 'app_attributes'
}, index=[0])

# Updating the data understanding dataframe:
data_und = pd.concat([data_und, data_und_update], axis=0, sort=False)
data_und.tail(1)

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class,category
0,num_words_desc,float64,700,[ 23. 490. 234. nan 183. 593. 660. 587. 626. ...,3,0.000164,numerical,app_attributes


#### P(X)

In [ ]:
df_train['num_words_desc'].describe()

count    18295.000000
mean       171.399344
std        151.119600
min          1.000000
25%         56.000000
50%        120.000000
75%        248.000000
max       2140.000000
Name: num_words_desc, dtype: float64

In [ ]:
share_miss_new = df_train.num_words_desc.isnull().sum()/len(df_train)
print(f'Share of missings: {share_miss_new:.4f}.')

Share of missings: 0.0002.


#### P(X|Y)

In [ ]:
df_train.groupby('class').describe()[['num_words_desc']]

num_words_desc                                                         
               count        mean         std  min   25%    50%    75%     max
class                                                                        
0             6088.0  150.166393  135.681855  1.0  53.0  103.0  204.0   771.0
1            12207.0  181.988859  157.194060  1.0  58.0  131.0  269.0  2140.0

<a id='share_malware_related_apps'></a>

### Share of malware related apps

In [ ]:
# Number of known related apps:
df_train['num_known_apps'] = df_train[['app_id', 'related_apps']].apply(lambda x: known_related_apps(data=df_train[df_train.app_id!=x['app_id']],
                                                                                                     related_apps=x['related_apps']),
                                                                        axis=1)
df_test['num_known_apps'] = df_test[['app_id', 'related_apps']].apply(lambda x: known_related_apps(data=df_train[df_train.app_id!=x['app_id']],
                                                                                                   related_apps=x['related_apps']),
                                                                      axis=1)

# Share of related apps that are known:
df_train['share_known'] = df_train['num_known_apps']/df_train['num_related_apps']
df_test['share_known'] = df_test['num_known_apps']/df_test['num_related_apps']

# Number of known related apps that are malwares:
df_train['num_known_malwares'] = df_train[['app_id', 'related_apps']].apply(lambda x: related_malwares(data=df_train[df_train.app_id!=x['app_id']],
                                                                                                       related_apps=x['related_apps']),
                                                                            axis=1)
df_test['num_known_malwares'] = df_test[['app_id', 'related_apps']].apply(lambda x: related_malwares(data=df_train[df_train.app_id!=x['app_id']],
                                                                                                     related_apps=x['related_apps']),
                                                                          axis=1)

# Share of known related apps that are malwares:
df_train['share_known_malwares'] = df_train['num_known_malwares']/df_train['num_known_apps']
df_test['share_known_malwares'] = df_test['num_known_malwares']/df_test['num_known_apps']

A saída de streaming foi truncada nas últimas 5000 linhas.
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in d

In [ ]:
new_var = 'num_known_apps'

# Describing the new feature:
data_und_update = pd.DataFrame(data={
    'feature': new_var, 'type': df_train.dtypes[new_var], 'n_unique': df_train[new_var].nunique(),
    'sample_values': str(list(df_train[new_var].unique()) if len(list(df_train[new_var].unique())) <= 10 else np.random.choice(list(df_train[new_var].unique()),
                                                                                                                               size=10,
                                                                                                                               replace=False)),
    'num_missings': df_train[new_var].isnull().sum(), 'share_missings': df_train[new_var].isnull().sum()/len(df_train),
    'var_class': 'numerical', 'category': 'app_attributes'
}, index=[0])

# Updating the data understanding dataframe:
data_und = pd.concat([data_und, data_und_update], axis=0, sort=False)
data_und.tail(1)

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class,category
0,num_known_apps,float64,5,"[0.0, 1.0, 2.0, 3.0, nan, 4.0]",484,0.026451,numerical,app_attributes


In [ ]:
new_var = 'share_known'

# Describing the new feature:
data_und_update = pd.DataFrame(data={
    'feature': new_var, 'type': df_train.dtypes[new_var], 'n_unique': df_train[new_var].nunique(),
    'sample_values': str(list(df_train[new_var].unique()) if len(list(df_train[new_var].unique())) <= 10 else np.random.choice(list(df_train[new_var].unique()),
                                                                                                                               size=10,
                                                                                                                               replace=False)),
    'num_missings': df_train[new_var].isnull().sum(), 'share_missings': df_train[new_var].isnull().sum()/len(df_train),
    'var_class': 'numerical', 'category': 'app_attributes'
}, index=[0])

# Updating the data understanding dataframe:
data_und = pd.concat([data_und, data_und_update], axis=0, sort=False)
data_und.tail(1)

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class,category
0,share_known,float64,7,"[0.0, 0.25, 0.5, 0.75, nan, 0.3333333333333333...",484,0.026451,numerical,app_attributes


In [ ]:
new_var = 'num_known_malwares'

# Describing the new feature:
data_und_update = pd.DataFrame(data={
    'feature': new_var, 'type': df_train.dtypes[new_var], 'n_unique': df_train[new_var].nunique(),
    'sample_values': str(list(df_train[new_var].unique()) if len(list(df_train[new_var].unique())) <= 10 else np.random.choice(list(df_train[new_var].unique()),
                                                                                                                               size=10,
                                                                                                                               replace=False)),
    'num_missings': df_train[new_var].isnull().sum(), 'share_missings': df_train[new_var].isnull().sum()/len(df_train),
    'var_class': 'numerical', 'category': 'app_attributes'
}, index=[0])

# Updating the data understanding dataframe:
data_und = pd.concat([data_und, data_und_update], axis=0, sort=False)
data_und.tail(1)

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class,category
0,num_known_malwares,float64,5,"[0.0, 1.0, 3.0, 2.0, nan, 4.0]",484,0.026451,numerical,app_attributes


In [ ]:
new_var = 'share_known_malwares'

# Describing the new feature:
data_und_update = pd.DataFrame(data={
    'feature': new_var, 'type': df_train.dtypes[new_var], 'n_unique': df_train[new_var].nunique(),
    'sample_values': str(list(df_train[new_var].unique()) if len(list(df_train[new_var].unique())) <= 10 else np.random.choice(list(df_train[new_var].unique()),
                                                                                                                               size=10,
                                                                                                                               replace=False)),
    'num_missings': df_train[new_var].isnull().sum(), 'share_missings': df_train[new_var].isnull().sum()/len(df_train),
    'var_class': 'numerical', 'category': 'app_attributes'
}, index=[0])

# Updating the data understanding dataframe:
data_und = pd.concat([data_und, data_und_update], axis=0, sort=False)
data_und.tail(1)

,feature,type,n_unique,sample_values,num_missings,share_missings,var_class,category
0,share_known_malwares,float64,7,"[nan, 1.0, 0.0, 0.5, 0.3333333333333333, 0.666...",10047,0.549076,numerical,app_attributes


#### P(X)

In [ ]:
df_train[['num_known_apps', 'share_known', 'num_known_malwares', 'share_known_malwares']].describe()

,num_known_apps,share_known,num_known_malwares,share_known_malwares
count,17814.000000,17814.000000,17814.000000,8251.000000
mean,0.732514,0.187493,0.440272,0.615582
std,0.961518,0.247452,0.782749,0.465083
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.250000,1.000000,1.000000
max,4.000000,1.000000,4.000000,1.000000


In [ ]:
share_miss_new = df_train.num_known_apps.isnull().sum()/len(df_train)
print(f'Share of missings of num_known_apps: {share_miss_new:.4f}.')

share_miss_new = df_train.share_known.isnull().sum()/len(df_train)
print(f'Share of missings of share_known: {share_miss_new:.4f}.')

share_miss_new = df_train.num_known_malwares.isnull().sum()/len(df_train)
print(f'Share of missings of num_known_malwares: {share_miss_new:.4f}.')

share_miss_new = df_train.share_known_malwares.isnull().sum()/len(df_train)
print(f'Share of missings of share_known_malwares: {share_miss_new:.4f}.')

Share of missings of num_known_apps: 0.0265.
Share of missings of share_known: 0.0265.
Share of missings of num_known_malwares: 0.0265.
Share of missings of share_known_malwares: 0.5491.


#### P(X|Y)

In [ ]:
df_train.groupby('class').describe()[['num_known_apps', 'share_known', 'num_known_malwares', 'share_known_malwares']]

num_known_apps                           ... share_known_malwares                    
               count      mean       std  min  ...                  25%  50%       75%  max
class                                          ...                                         
0             6040.0  0.928808  1.025861  0.0  ...                  0.0  0.0  0.666667  1.0
1            11774.0  0.631816  0.910546  0.0  ...                  1.0  1.0  1.000000  1.0

[2 rows x 32 columns]

In [ ]:
df_test.groupby('class').describe()[['num_known_apps', 'share_known', 'num_known_malwares', 'share_known_malwares']]

num_known_apps                           ... share_known_malwares                    
               count      mean       std  min  ...                  25%  50%       75%  max
class                                          ...                                         
0             2957.0  0.909706  1.037995  0.0  ...                  0.0  0.0  0.666667  1.0
1             5819.0  0.608696  0.890344  0.0  ...                  1.0  1.0  1.000000  1.0

[2 rows x 32 columns]

<a id='nlp'></a>

### Natural language processing

In [ ]:
### Possibilities:
  ### BOW and TF-IDF.
  ### LLR for selecting risky words.
  ### Other approaches for selecting risky words.
    ### For instance: treat texts, create binary variables for each word, drop irrelevant words (dummy variable with variance less than 0.01), train
    ### bivariate models (outcome variable against each dummy variable), select only those words whose model has ROC-AUC calculated through K-folds CV
    ### higher than 0.5.
    ### Alternatively, instead of applying supervised learning models, calculate, for each dummy variable, differences in means of the dummy variable with
    ### y = 1 against y = 0, or then differences in means of y with the dummy equals to 1 against the dummy equals to 0.

<a id='export_data'></a>

### Exporting training and test data

In [ ]:
if EXPORT:
  data_und.to_csv('../data/features.csv', index=False)
  df_train.to_csv('../data/training_data.csv', index=False)
  df_test.to_csv('../data/test_data.csv', index=False)